# Test Develop Mock Circuit

In [1]:
import numpy as np
import pandas as pd
from neuro_dmt.models.bluebrain.circuit.mock.cell import Cell

In [2]:
from neuro_dmt.models.bluebrain.circuit.geometry import Position
p = Position(X = 1., Y = 1., Z = 1.)
print(p.as_tuple)
print(p.value)

random_positions =\
    Position.sample(
        (np.array((-1., -1., -1.)), np.array((1., 1., 1.))),
        n = 3)
print(random_positions)

(1.0, 1.0, 1.0)
[1. 1. 1.]
[[ 6.82994963e-04 -4.23524410e-01  6.88798119e-01]
 [ 4.98029117e-01  5.97937302e-01  6.18600594e-01]
 [-5.71178978e-01 -2.01710791e-01 -3.15901389e-04]]


In [3]:
example_cell =\
    Cell(
        layer=1,
        position=Position(X=1., Y=2., Z=3.),
        mtype="L23_BTC")
example_cell.as_dict

{'region': 'brain',
 'layer': 1,
 'nucleus': 'not-defined',
 'mtype': 'L23_BTC',
 'etype': 'not-defined',
 'morph_class': 'not-defined',
 'synapse_class': 'INH',
 'x': 1.0,
 'y': 2.0,
 'z': 3.0}

In [4]:
from neuro_dmt.models.bluebrain.circuit.mock.test_develop\
    import circuit_composition, circuit_connectivity
circuit_composition.cell_density.head()

mean  std
mtype  layer                  
L1_DAC 1      3094.333333  0.0
       2         0.000000  0.0
       3         0.000000  0.0
       4         0.000000  0.0
       5         0.000000  0.0

In [5]:
from neuro_dmt.models.bluebrain.circuit.mock import CircuitBuilder
test_circuit_builder =\
    CircuitBuilder(
        composition=circuit_composition,
        connectivity=circuit_connectivity)
test_circuit_builder.get_cell_density( mtype="L1_DAC", layer=1 )

,,mean,std
mtype,layer,,
L1_DAC,1,3094.333333,0.0


In [6]:
print(circuit_composition.cell_density.loc["L1_DAC"])
print(test_circuit_builder.get_number_cells(1, "L1_DAC"))

              mean  std
layer                  
1      3094.333333  0.0
2         0.000000  0.0
3         0.000000  0.0
4         0.000000  0.0
5         0.000000  0.0
6         0.000000  0.0
23


In [7]:
circuit_cells = test_circuit_builder.get_cells()

In [8]:
import pandas as pd
circuit_cells_df =\
    pd.DataFrame([
        cell.as_dict for cell in circuit_cells])
print("number of cells {}".format(circuit_cells_df.shape[0]))
print(circuit_cells_df.head())

number of cells 12237
         etype  layer  morph_class   mtype      nucleus region synapse_class  \
0  not-defined      1  not-defined  L1_DAC  not-defined  brain           INH   
1  not-defined      1  not-defined  L1_DAC  not-defined  brain           INH   
2  not-defined      1  not-defined  L1_DAC  not-defined  brain           INH   
3  not-defined      1  not-defined  L1_DAC  not-defined  brain           INH   
4  not-defined      1  not-defined  L1_DAC  not-defined  brain           INH   

            x            y           z  
0   31.249079  1834.972332  180.579263  
1  188.879507  1751.207639   92.751844  
2  206.607160  1790.535389  196.326716  
3   42.006491  1734.946362   72.979289  
4  190.779082  1796.398099  235.769273  


In [9]:
from neuro_dmt.models.bluebrain.circuit.mock.synapse import Synapse
s = Synapse(pre_gid=1, post_gid=2)
s.as_dict

{'post_gid': 2, 'pre_gid': 1}

In [10]:
cell_collection = test_circuit_builder.get_cell_collection()
cell_collection.get(1, "mtype")

'L1_DAC'

In [11]:
cell_collection.get([1,2], properties=["mtype", "etype"])

,mtype,etype
1,L1_DAC,not-defined
2,L1_DAC,not-defined


In [12]:
from neuro_dmt.models.bluebrain.circuit.mock.circuit import MockCircuit
mock_circuit =\
    MockCircuit.build(
        circuit_composition,
        circuit_connectivity)
mock_conn =\
    mock_circuit.connectome

In [13]:
mock_cells = mock_circuit.cells
mock_cells.get([1, 3000], "mtype")

1       L1_DAC
3000     L4_PC
Name: mtype, dtype: object

In [14]:
mock_conn = mock_circuit.connectome
print(mock_conn.afferent_gids(2))
print(mock_conn.efferent_gids(14))

[    4     6    12 ... 12222 12230 12232]
[    6     7     9 ... 12210 12215 12230]


In [16]:
post_gid = 0
pre_gid = 16
print(mock_conn._get_pair_synapse_count(pre_gid, post_gid))
print(mock_conn.afferent_adjacency[post_gid][0:10,:])
print(mock_conn._get_synapses(pre_gid, post_gid))
print(mock_conn.pair_synapses(pre_gid, post_gid))
#print(mock_conn.pair_synapses(pre_gid, 1))

4
[[ 4  4]
 [11  4]
 [16  4]
 [19  4]
 [20  5]
 [22  5]
 [23  4]
 [24  5]
 [26  7]
 [27  3]]
                                pre_gid  post_gid  synapse_index
pre_gid post_gid synapse_index                                  
16      0        0                   16         0              0
                 1                   16         0              1
                 2                   16         0              2
                 3                   16         0              3
[(16, 0, 0) (16, 0, 1) (16, 0, 2) (16, 0, 3)]


In [17]:
gid = 10
aff_gids = mock_conn.afferent_gids(gid)
print(len(aff_gids))
aff_syns = mock_conn.afferent_synapses(gid)
print(len(aff_syns))

1957
9751


In [19]:
gid = 10
eff_gids = mock_conn.efferent_gids(gid)
print(len(eff_gids))
eff_syns = mock_conn.efferent_synapses(gid)
#eff_syns = mock_conn._get_properties(
 #   mock_conn._get_synapses(pre_gid=gid),
  #  properties=None)
print(len(eff_syns))

2032
10316


In [ ]:
print(afferent_synapses.head())
print(2 in afferent_synapses.index)
afferent_synapses.xs(
    11, level="pre_gid")

In [ ]:
mock_conn._synapses = mock_conn.empty_synapse_holder()
mock_conn.

In [ ]:
post_gid = 0
synapses = mock_conn.empty_synapse_holder()
synapses = synapses.append(
    mock_conn._read_synapses(0, 0))
for pre_gid in mock_conn.afferent_gids(post_gid):
    synapses = synapses.append(
        mock_conn._read_synapses(pre_gid, post_gid))
print(synapses)

In [ ]:
ss = [(pre_gid, mock_conn._synapses_slice(pre_gid, post_gid))
      for pre_gid in mock_conn.afferent_gids(post_gid)]
ss[:10]

In [ ]:
df = pd.DataFrame(dict(x=np.arange(100), y=np.arange(150, 250)))\
       .set_index(["x", "y"], drop=False)
print(df.xs(2, level="x"))
print(df.xs((1,150), level=("x", "y")))

In [ ]:
post_gid = 0
egdf = pd.concat([
    pd.DataFrame({
        "pre_gid": pre_gid,
        "post_gid": post_gid,
        "synapse_index": np.arange(count, dtype=np.int32)})
    for pre_gid, count in mock_conn.afferent_adjacency[post_gid]])\
    .set_index(
        ["pre_gid", "post_gid"],
        drop=False)
print(np.arange(0))

In [ ]:
pd.DataFrame({
    "pre_gid": 1,
    "post_gid": 2,
    "synapse_index": np.arange(0)})\
    .set_index(
        ["pre_gid", "post_gid"],
        drop=False)